<h1> <a href=http://www.datascience-paris-saclay.fr/>Paris Saclay Center for Data Science</a> </h1>

<h2> RAMP on qualitative and quantitative non-invasive monitoring of anti-cancer drugs </h2>

<i>Camille Marini (LTCI/CNRS), Alex Gramfort (LTCI/Télécom ParisTech), Sana Tfaili (Lip(Sys)²/UPSud), Laetitia Le (Lip(Sys)²/UPSud), Mehdi Cherti (LAL/CNRS), Balázs Kégl (LAL/CNRS)</i>

## Drug Spectra: Fall 2016 - Arpa MUKHERJEE

## Exploratory data analysis

In [1]:
%matplotlib inline
import os
import glob
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import pandas as pd

### Loading the data

In [2]:
# Load training data

data = pd.read_csv('train.csv')

y_df = data[['molecule', 'concentration']]
X_df = data.drop(['molecule', 'concentration'], axis=1)
spectra = X_df['spectra'].values                                        
spectra = np.array([np.array(dd[1:-1].split(',')).astype(float) for dd in spectra])    
X_df['spectra'] = spectra.tolist()

# Loading wavenumbers
freqs = pd.read_csv('freq.csv')
freqs = freqs['freqs'].values

# Types of molecules
types = np.unique(y_df['molecule'].values)
# print(types)

# Target for classification
molecule = y_df['molecule'].values
# Target for regression
concentration = y_df['concentration'].values
# "Raw" features
X = spectra

#### Extra Data Stats

In [ ]:
# This was just to look at descriptive stats for all the data sources to get an idea based on pandas.df.describe

def describe_adj(df):
    objects = []
    numerics = []
    for c in df:
        if (df[c].dtype == object):
            objects.append(c)
        else:
            numerics.append(c)

    return df[objects].describe(), df[numerics].describe()

In [ ]:
describe_adj(data)

#### Extra Raman spectra viz

In [ ]:
# Plot before and after filter which I used for the feature extraction.
from scipy.signal import savgol_filter

spectra1 = np.array([np.array(dd) for dd in X_df['spectra']])
spectra_sgsmooth = savgol_filter(spectra1, 15, 3) # window size 5, polynomial order 3


bx = plt.subplot('211')
bx.set_title("Original Spectra")
plt.plot(freqs, spectra.T)
bx2 = plt.subplot('212')
bx2.set_title("Savitsky Golay Filter")
plt.plot(freqs, spectra_sgsmooth.T)
plt.tight_layout()
plt.show()


#### Extra Concentration viz

In [ ]:
# To see the frequency of concentration by molecule in a different form and to get an understanding of the distribution.
# Helps to inform which regressor to choose.

def stbars(df, conc, mtype):

    mtype_list = data[mtype].unique()
    conc_list = data[conc].unique()
  
    # outer:
    outdict = {}
    for m in mtype_list:
        m_df = df[df[mtype]==m]
        m_count = m_df.groupby(conc).count() 
        
        # inner:
        indict ={}
        for x in conc_list:
            indict[x] = (m_count.loc[x].values[0] if x in m_count.index.values else 0)
        outdict[m] = indict
        
    return pd.DataFrame(outdict)

In [ ]:
df_mol_conc = stbars(data,"concentration","molecule")
colors = ['#624ea7', 'g', 'yellow', 'k', 'maroon']
ax = df_mol_conc.plot.barh(stacked=True, figsize=[9,4], color=colors)
ax.set_title("Concentration per molecule")
ax.set_xlabel("Count")
ax.set_ylabel("Concentrations")
print(df_mol_conc) # note to self: maybe use this data to calculate weights for RFR

### Feature extractor for classification

Since this was spectral data, I only thought to apply a filter, and after some research decided to choose the Savitzky-Golay filter to smooth the data as it does not distort the signal too much. I found it made a slight improvement compared to the original. I read in theory for smoothing effect to appear, the polynomial order should be less than the window size, and I chose a ratio of 1/5 (3 vs 15) after some testing. I tested adding PCA also with the filter but it seemed to worsen the classifier results, so I decided to keep it out. 

In [11]:
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter

class FeatureExtractorClf(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_df):
        pass
    
    def transform(self, X_df):
        XX = np.array([np.array(dd) for dd in X_df['spectra']])
        XX_sgsmooth = savgol_filter(XX, 15, 3) # window size 15, polynomial order 3
        return XX_sgsmooth
    

### Classification: predicting the molecule type

I first decided to choose the classifier SVM with its default properties instead of logistic regression because the number of features is small compared to the number of training examples. At first I had included PCA with SVC, and then decided to switch to Stochastic Gradient Descent Classifier in the pipeline, and this coupled with SVC with kernel set to linear really improved the results. Additionally, adjusting the penalty parameter with SVC, C, to higher results, helped to further minimize error.

In [12]:
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
 
 
class Classifier(BaseEstimator):
    def __init__(self):
        self.clf = Pipeline([
            ('sgd', SGDClassifier(loss='hinge', penalty='l2')),
            ('clf', SVC(kernel='linear', C = 100000, probability=True)),
        ])
 
    def fit(self, X, y):
        self.clf.fit(X, y)
 
    def predict(self, X):
        return self.clf.predict(X)
 
    def predict_proba(self, X):
        return self.clf.predict_proba(X)

In [13]:
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

labels = np.array(['A', 'B', 'Q', 'R'])

def train_test_model_clf(X_df, y_df, skf_is, FeatureExtractor, Classifier):
    train_is, test_is = skf_is
    X_train_df = X_df.iloc[train_is].copy()                                  
    y_train_df = y_df.iloc[train_is].copy()
    y_train_clf = y_train_df['molecule'].values
    X_test_df = X_df.iloc[test_is].copy()                                    
    y_test_df = y_df.iloc[test_is].copy() 
    y_test_clf = y_test_df['molecule'].values 
    # Feature extraction
    fe_clf = FeatureExtractor()
    fe_clf.fit(X_train_df, y_train_df)
    X_train_array_clf = fe_clf.transform(X_train_df)
    X_test_array_clf = fe_clf.transform(X_test_df)
    # Train
    clf = Classifier()
    clf.fit(X_train_array_clf, y_train_clf)
    # Test 
    y_proba_clf = clf.predict_proba(X_test_array_clf)                        
    y_pred_clf = labels[np.argmax(y_proba_clf, axis=1)]                      
    error = 1 - accuracy_score(y_test_clf, y_pred_clf)                       
    print('error = %s' % error)                                                                            
    print('classification report:\n %s' % classification_report(y_test_clf, y_pred_clf))
    print('confusion matrix:\n %s' % confusion_matrix(y_test_clf, y_pred_clf))


skf = ShuffleSplit(n_splits=2, test_size=0.2, random_state=57)  
skf_is = list(skf.split(X_df))[0]

train_test_model_clf(X_df, y_df, skf_is, FeatureExtractorClf, Classifier)

error = 0.03
classification report:
              precision    recall  f1-score   support

          A       0.97      0.97      0.97        63
          B       0.95      0.93      0.94        45
          Q       1.00      0.97      0.99        40
          R       0.96      1.00      0.98        52

avg / total       0.97      0.97      0.97       200

confusion matrix:
 [[61  2  0  0]
 [ 1 42  0  2]
 [ 1  0 39  0]
 [ 0  0  0 52]]


### Feature extractor for regression

I started with PCA at its default setting since we have both molecule types and spectra data with some multicollinearity. The main adjustment here I made is to set it to randomized PCA which does not center the data as in the regular SVD used by PCA and seemed to offer slightly better results.


In [14]:
import numpy as np
from sklearn.decomposition import PCA
 
labels = np.array(['A', 'B', 'Q', 'R'])
 
class FeatureExtractorReg(object):
    def __init__(self):
        self.pca = PCA(n_components=10, svd_solver = 'randomized') # Add randomized PCA
 
    def fit(self, X_df, y):
        XX = np.array([np.array(dd) for dd in X_df['spectra']])  
        XX -= np.mean(XX, axis=1)[:, None]                                     
        XX /= np.sqrt(np.sum(XX ** 2, axis=1))[:, None]
        self.pca.fit(XX)
    
    def transform(self, X_df):                                                   
        XX = np.array([np.array(dd) for dd in X_df['spectra']])                  
        XX -= np.mean(XX, axis=1)[:, None]                                     
        XX /= np.sqrt(np.sum(XX ** 2, axis=1))[:, None]
        XX = self.pca.transform(XX) 
        XX = np.concatenate([XX, X_df[labels].values], axis=1)                 
        return XX   
       

### Regression: predicting the concentration

I left PCA in the pipeline at first, and I experimented with both Gradient Boosting Regressor vs AdaBoost Regressor, with the latter offering a slightly better score. I decided to keep AdaBoost Regressor and pair with Random Forest Regressor because the concentration data for each molecule type is very unevenly distributed and I thought the Random Forest may learn better, and the pairing with AdaBoost seems to give good performance in general. I experimented with several parameters of Random Forest but could not seem to improve the results too much. My next approach would be to compute and add in weights from the training data for each molecule at each concentration and implement the 'min_weight_leaf' feature of RFR.. 

In [38]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
import numpy as np
 
class Regressor(BaseEstimator):                                                  
    def __init__(self):                                                          
        self.n_components = 10                                                  
#        self.n_estimators = 80                                                   
#        self.learning_rate = 0.2                                              
        self.list_molecule = ['A', 'B', 'Q', 'R']   
        self.dict_reg = {}                                                       
        for mol in self.list_molecule:  
            self.dict_reg[mol] = Pipeline([('pca', PCA(n_components=self.n_components, random_state=42)),
              ('reg', SVC(probability=True,C=100000,gamma=10.0,kernel='linear'))])
            
#         self.dict_reg['R'] = AdaBoostClassifier(
#          RandomForestClassifier(criterion='entropy', n_estimators=self.n_estimators), n_estimators=self.n_estimators, 
#         learning_rate=self.learning_rate, algorithm="SAMME")
         
    def fit(self, X, y):
        self.viewed = {}
        for i, mol in enumerate(self.list_molecule):
            ind_mol = np.where(np.argmax(X[:, -4:], axis=1) == i)[0]
            XX_mol = X[ind_mol]
            y_mol = y[ind_mol]
            self.dict_reg[mol].fit(XX_mol, y_mol.astype(str))
            self.viewed[mol] = list(set(y_mol))
    

    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        for i, mol in enumerate(self.list_molecule):
            ind_mol = np.where(np.argmax(X[:, -4:], axis=1) == i)[0]
            XX_mol = X[ind_mol]
            y_pred[ind_mol] =(self.dict_reg[mol].predict(XX_mol).astype(float))
        return y_pred
 

In [39]:
from sklearn.model_selection import ShuffleSplit
import warnings
warnings.filterwarnings('ignore')

def mare_score(y_true, y_pred):                                                  
    return np.mean(np.abs((y_true - y_pred) / y_true)) 

def train_test_model(X_df, y_df, skf_is, FeatureExtractorClf, Classifier, FeatureExtractorReg, Regressor):
    train_is, test_is = skf_is
    X_train_df = X_df.iloc[train_is].copy()                                  
    y_train_df = y_df.iloc[train_is].copy()                                  
    X_test_df = X_df.iloc[test_is].copy()                                    
    y_test_df = y_df.iloc[test_is].copy()                                    
    y_train_clf = y_train_df['molecule'].values                              
    y_train_reg = y_train_df['concentration'].values                         
    y_test_clf = y_test_df['molecule'].values                                
    y_test_reg = y_test_df['concentration'].values                           

    # Classification
    fe_clf = FeatureExtractorClf()                     
    fe_clf.fit(X_train_df, y_train_df)                                       
    X_train_array_clf = fe_clf.transform(X_train_df)                         
    X_test_array_clf = fe_clf.transform(X_test_df)                           
                                                                                 
    clf = Classifier()                                            
    clf.fit(X_train_array_clf, y_train_clf)                                  
    y_proba_clf = clf.predict_proba(X_test_array_clf)                        
    y_pred_clf = labels[np.argmax(y_proba_clf, axis=1)]                      
    error = 1 - accuracy_score(y_test_clf, y_pred_clf)                       
    print('error = ', error)
    
    # Regression
    fe_reg = FeatureExtractorReg()                     
    for i, label in enumerate(labels):
        # For training, we use 
        X_train_df.loc[:, label] = (y_train_df['molecule'] == label)         
        X_test_df.loc[:, label] = y_proba_clf[:, i]                          
    fe_reg.fit(X_train_df, y_train_reg)                                      
    X_train_array_reg = fe_reg.transform(X_train_df)                         
    X_test_array_reg = fe_reg.transform(X_test_df)                           
                                                                                 
    reg = Regressor()                                              
    reg.fit(X_train_array_reg, y_train_reg)                               
    y_pred_reg = reg.predict(X_test_array_reg)
    mare = mare_score(y_test_reg, y_pred_reg)
    print('mare = ', mare)                
    print('combined error = ', 2. / 3 * error + 1. / 3 * mare)


skf = ShuffleSplit(n_splits=2, test_size=0.2, random_state=57) 
skf_is = list(skf.split(X_df))[0]

train_test_model(X_df, y_df, skf_is, FeatureExtractorClf, Classifier, FeatureExtractorReg, Regressor)

error =  0.04
mare =  0.0421071428571
combined error =  0.0407023809524
